In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import os

In [ ]:
sns.set(style="whitegrid")

conn = sqlite3.connect("C:/Users/Admin/OneDrive/Escritorio/JonasLuna_TPintegrador/CLIMA/datos/analisis.db")
df_ciudades = pd.read_sql_query("SELECT * FROM resumen_clima", conn)
df_global = pd.read_sql_query("SELECT * FROM resumen_global", conn)

conn.close()


,Temperatura promedio (°C),Sensacion termica promedio (°C),Humedad promedio (%),Diferencia termica promedio (°C)
0,17.73,17.42,71.33,0.49
1,18.93,18.87,75.00,0.31
2,27.32,28.35,69.20,1.03
3,12.25,11.11,69.13,1.14
4,16.82,15.77,46.47,1.05
5,16.58,15.64,56.73,0.95
6,13.56,12.36,50.13,1.21
7,18.79,18.69,75.00,0.21
8,19.60,18.66,39.87,0.94
9,19.72,19.02,48.47,0.71


In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(data=df_ciudades, x="Ciudad", y="Temperatura promedio (°C)", palette="coolwarm")
plt.title("Temperatura promedio por ciudad")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

ValueError: Could not interpret value `Ciudad` for `x`. An entry with this name does not appear in `data`.

<Figure size 1000x600 with 0 Axes>